## Train Primary Shower Networks

written by Isobel Mawby (i.mawby1@lancaster.ac.uk)

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Imports
</div>

In [ ]:
import sys
import os
sys.path.insert(0, os.getcwd()[0:len(os.getcwd()) - 11])
sys.path.insert(1, os.getcwd()[0:len(os.getcwd()) - 11] + '/Metrics')

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import sys
import itertools

import Models
import TrainingMetrics

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Put the path to the primary shower training file (created by WritePrimaryTierFile.ipynb with isTrackMode == False) and set ouput file name
</div>

In [ ]:
trainFileName = sys.path[0] + '/files/hierarchy_TRAIN_shower.npz'
classifierModelPath = sys.path[0] + '/models/PandoraNet_Hierarchy_DUNEFD_HD_S_Class_v014_15_00'

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Set hyperparameters
</div>

In [ ]:
N_EPOCHS = 5
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
DROPOUT_RATE = 0.5

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Get data from file
</div>

In [ ]:
data = np.load(trainFileName)

# Variables
variables_train = data['variables_train']
variables_test = data['variables_test']
variables_train = variables_train.astype(np.float32)
variables_test = variables_test.astype(np.float32)
# Truth
isTruePrimaryLink_train = data['isTruePrimaryLink_train']
isTruePrimaryLink_test = data['isTruePrimaryLink_test']
# Training cut
trainingCutDCA_train = data['trainingCutDCA_train']
trainingCutDCA_test = data['trainingCutDCA_test']

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Set multiplicity variables
</div>

In [ ]:
nVariables = variables_train.shape[1]

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Check shapes
</div>

In [ ]:
print('variables_train.shape:', variables_train.shape)
print('variables_test.shape:', variables_test.shape)
print('trainingCutDCA_train.shape:', trainingCutDCA_train.shape)
print('trainingCutDCA_test.shape:', trainingCutDCA_test.shape)
print('isTruePrimaryLink_train.shape:', isTruePrimaryLink_train.shape)
print('isTruePrimaryLink_test.shape:', isTruePrimaryLink_test.shape)
print('')
print('ntrain:', variables_train.shape[0])
print('ntest:', variables_test.shape[0])

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Apply training cut mask
</div>

In [ ]:
# training cut threshold
MAX_TRAINING_CUT_DCA = 50.0

######################
# training set first
######################
# Make mask
passTrainingCutDCA_train = trainingCutDCA_train < MAX_TRAINING_CUT_DCA
passTrainingCuts_train = passTrainingCutDCA_train

# Mask the 1D variables... shape=(nEntries, )
isTruePrimaryLink_train = isTruePrimaryLink_train[passTrainingCuts_train]

# Mask the variable... shape=(nEntries, nVariables)
variables_train = variables_train[[[entry] * nVariables for entry in passTrainingCuts_train]].reshape(-1, nVariables)

######################
# now test set
######################
# Make mask
passTrainingCutDCA_test = trainingCutDCA_test < MAX_TRAINING_CUT_DCA
passTrainingCuts_test = passTrainingCutDCA_test

# Mask the 1D variables... shape=(nEntries, )
isTruePrimaryLink_test = isTruePrimaryLink_test[passTrainingCuts_test]

# Mask the variable... shape=(nEntries, nVariables)
variables_test = variables_test[[[entry] * nVariables for entry in passTrainingCuts_test]].reshape(-1, nVariables)

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Check shapes after training cut application
</div>

In [ ]:
print('variables_train.shape:', variables_train.shape)
print('variables_test.shape:', variables_test.shape)
print('isTruePrimaryLink_train.shape:', isTruePrimaryLink_train.shape)
print('isTruePrimaryLink_test.shape:', isTruePrimaryLink_test.shape)
print('')
print('ntrain:', variables_train.shape[0])
print('ntest:', variables_test.shape[0])

<div class="alert alert-block alert-info" style="font-size: 18px;">
   Define class weights
</div>

In [ ]:
nTrue_final = np.count_nonzero(isTruePrimaryLink_train == 1)
nBackground_final = np.count_nonzero(isTruePrimaryLink_train == 0)
maxLinks = max(nTrue_final, nBackground_final)

classWeights_final = {0: maxLinks/nBackground_final, 1: maxLinks/nTrue_final}

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Prepare Dataset objects
</div>

In [ ]:
loader_train = DataLoader(list(zip(variables_train, isTruePrimaryLink_train)), shuffle=True, batch_size=BATCH_SIZE)
loader_test = DataLoader(list(zip(variables_test, isTruePrimaryLink_test)), shuffle=True, batch_size=BATCH_SIZE)

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Define model
</div>

In [ ]:
model = Models.PrimaryTrackShowerModel(nVariables, dropoutRate=DROPOUT_RATE)

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Define loss functions for training to implement custom weighting
</div>

In [ ]:
def loss_function_classifier(pred, targets, classWeights) :
    # Loss function
    loss_func = torch.nn.BCELoss()    
    # Do weighting
    weight = torch.ones(targets.shape)
    weight[targets > 0.5] = classWeights_final[1]
    weight[targets < 0.5] = classWeights_final[0]
    loss_func.weight = weight
    # Calc loss
    targets = targets.to(torch.float64)
    pred = pred.type(torch.float64)
    loss = loss_func(pred, targets)
       
    return loss

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Training/validation loop functions.
</div>

In [ ]:
def RunTrainingLoop(inputs, targets, classifier_model, classWeights_classifier) : 
    # Get predictions
    preds = classifier_model(inputs).reshape(-1)
    # Get loss
    loss = loss_function_classifier(preds, targets, classWeights_classifier)
    return loss

def RunValidationLoop(inputs, targets, classifier_model, classWeights_classifier, linkMetrics) : 
    # Get predictions
    preds = classifier_model(inputs).reshape(-1)
    # Get loss
    loss = loss_function_classifier(preds, targets, classWeights_classifier)
    
    linkMetrics.classifier_metrics.Fill(loss, preds, targets)   

<div class="alert alert-block alert-info" style="font-size: 18px;">
   Training/testing loops
</div>

In [ ]:
# Optimiser
optimiser = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Put here some metrics
epochs_metrics = []
training_link_metrics = []
testing_link_metrics = []

for epoch in range(N_EPOCHS):
    
    # Begin training mode
    model.train()
                         
    for inputs, targets in loader_train:  
    
        # Skip incomplete batches
        if (inputs.shape[0] != BATCH_SIZE) :
            continue       
            
        loss = RunTrainingLoop(inputs, targets, model, classWeights_final)
        
        # Update model parameters
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()   
        
    with torch.no_grad():
        
        # Begin testing mode
        model.eval()
        
        # Initialise metrics        
        linkMetrics_train = TrainingMetrics.LinkMetrics(0)
        linkMetrics_test = TrainingMetrics.LinkMetrics(0)
                        
        # Iterate in batches over the training dataset.                        
        for inputs_train, targets_train in loader_train:  

            # Skip incomplete batches
            if (inputs_train.shape[0] != BATCH_SIZE) :
                continue        

            # Get predictions
            RunValidationLoop(inputs_train, targets_train, model, classWeights_final, linkMetrics_train)            
                        
        # Iterate in batches over the testing dataset.                        
        for inputs_test, targets_test in loader_test:  

            # Skip incomplete batches
            if (inputs_test.shape[0] != BATCH_SIZE) :
                continue        

            # Get predictions
            RunValidationLoop(inputs_test, targets_test, model, classWeights_final, linkMetrics_test)    
            
        epochs_metrics.append(epoch)            
    
    ##########################
    # Calc metrics for epoch 
    ##########################   
    # Find threshold
    optimal_threshold_train, maximum_accuracy_train = TrainingMetrics.calculate_accuracy(linkMetrics_train)
    optimal_threshold_test, maximum_accuracy_test = TrainingMetrics.calculate_accuracy(linkMetrics_test)

    # Calculate metrics
    linkMetrics_train.Evaluate(optimal_threshold_train)
    linkMetrics_test.Evaluate(optimal_threshold_test)
    
    # Add to our lists
    training_link_metrics.append(linkMetrics_train)
    testing_link_metrics.append(linkMetrics_test) 
    
   # Do some prints
    print('----------------------------------------')
    print('Epoch:', epoch)
    print('----------------------------------------')
    print('training_classification_loss:', round(linkMetrics_train.classifier_metrics.av_loss, 2))
    print('----')
    print('optimal_threshold_train:', optimal_threshold_train)
    print('accuracy_train:', str(round(maximum_accuracy_train.item(), 2)) +'%')
    print('positive_as_positive_fraction_train:', str(round(linkMetrics_train.classifier_metrics.pos_as_pos_frac * 100.0, 2)) + '%')
    print('positive_as_negative_fraction_train:', str(round(linkMetrics_train.classifier_metrics.pos_as_neg_frac * 100.0, 2)) + '%')
    print('negative_as_negative_fraction_train:', str(round(linkMetrics_train.classifier_metrics.neg_as_pos_frac * 100.0, 2)) + '%')
    print('negative_as_positive_fraction_train:', str(round(linkMetrics_train.classifier_metrics.neg_as_neg_frac * 100.0, 2)) + '%')
    print('----')
    print('testing_classification_loss:', round(linkMetrics_test.classifier_metrics.av_loss, 2))
    print('----')
    print('optimal_threshold_test:', optimal_threshold_test)
    print('accuracy_test:', str(round(maximum_accuracy_test.item(), 2)) +'%')
    print('positive_as_positive_fraction_test:', str(round(linkMetrics_test.classifier_metrics.pos_as_pos_frac * 100.0, 2)) + '%')
    print('positive_as_negative_fraction_test:', str(round(linkMetrics_test.classifier_metrics.pos_as_neg_frac * 100.0, 2)) + '%')
    print('negative_as_negative_fraction_test:', str(round(linkMetrics_test.classifier_metrics.neg_as_pos_frac * 100.0, 2)) + '%')
    print('negative_as_positive_fraction_test:', str(round(linkMetrics_test.classifier_metrics.neg_as_neg_frac * 100.0, 2)) + '%')
    print('----')    
    
    TrainingMetrics.plot_scores_classifier(linkMetrics_train, linkMetrics_test)

<div class="alert alert-block alert-info" style="font-size: 18px;">
   Plot metrics associated with training 
</div>

In [ ]:
TrainingMetrics.plot_classifier_loss_evolution(epochs_metrics, training_link_metrics, testing_link_metrics, 'Loss - classifier')
TrainingMetrics.plot_edge_rate(epochs_metrics, training_link_metrics, testing_link_metrics, True)
TrainingMetrics.plot_edge_rate(epochs_metrics,  training_link_metrics, testing_link_metrics, False)

<div class="alert alert-block alert-info" style="font-size: 18px;">
   Show ROC curve and confusion matrices, for the latter you can decide the threshold cut used
</div>

In [ ]:
with torch.no_grad():
    # Begin testing mode
    model.eval()
    
    pred_final_test = model(torch.tensor(variables_test))
    
    pos_scores_final_test = np.array(pred_final_test.tolist())[isTruePrimaryLink_test == 1]
    neg_scores_final_test = np.array(pred_final_test.tolist())[isTruePrimaryLink_test == 0]
    
    TrainingMetrics.plot_roc_curve(torch.tensor(pos_scores_final_test), torch.tensor(neg_scores_final_test))
    TrainingMetrics.draw_confusion_with_threshold(pred_final_test, isTruePrimaryLink_test, 0.5)

<div class="alert alert-block alert-info" style="font-size: 18px;">
   Save the model
</div>

In [ ]:
sm = torch.jit.script(model)
sm.save(f"{classifierModelPath}.pt")
torch.save(model.state_dict(), f"{classifierModelPath}.pkl")